## SageMaker Model Registry 
haimtran 01/06/2023

In [ ]:
# !python -m pip install sagemaker 
# !python -m pip install boto3

In [ ]:
import json
import sagemaker 
import boto3

In [ ]:
role = sagemaker.get_execution_role()
session = sagemaker.Session()
bucket = session.default_bucket()

In [ ]:
sagemaker_client = boto3.client("sagemaker")

## Create a Model 

In [ ]:
from sagemaker.model import Model
from sagemaker.model_metrics import ModelMetrics
from sagemaker.image_uris import retrieve

In [ ]:
image_uri = retrieve(
    framework="pca",
    region=session._region_name, 
    instance_type="ml.m4.xlarge",
    version="0.23-1"
)

In [ ]:
image_uri

In [ ]:
model = Model(
    image_uri=image_uri,
    model_data="s3://sagemaker-ap-southeast-1-394599967678/TrainingJobName27a61f71-876d-4d88-a39f-487cd1d1ebd4/output/model.tar.gz",
    role=role,
)

In [ ]:
# model.create(instance_type="ml.m4.xlarge")

## Create a Model Package Group 

In [ ]:
model_package_group_input_dict = {
    "ModelPackageGroupName": "ecg-model-group", 
    "ModelPackageGroupDescription": "develop model for ECG"
}

In [ ]:
# response = sagemaker_client.create_model_package_group(
#  **model_package_group_input_dict 
# )

In [128]:
# print(json.dumps(response, indent=4))

## Register a Model to a Model Group 

option 1. mode.create(), run model.register()

In [ ]:
# model.register(
#     model_package_group_name="ecg-model-group",
#     content_types=["text/csv"], 
#     inference_instances=["ml.m5.xlarge"], 
#     transform_instances=["ml.m5.xlarge"], 
#     response_types=["text/csv"], 
#     customer_metadata_properties={"ModelType": "PCA"}, 
#     # model_metrics=None,
#     approval_status="PendingManualApproval"
# )

option 2. using boto3 sagemaker client 

In [ ]:
# create_model_package_input_dict = {
#     "ModelPackageGroupName": "ecg-model-group",
#     "ModelPackageDescription": "", 
#     "ModelApprovalStatus" : "PendingManualApproval",
#     "InferenceSpecification": {
#         "Containers": [
#             {
#                 "Image": image_uri,
#                 "ModelDataUrl": "s3://sagemaker-ap-southeast-1-394599967678/TrainingJobName27a61f71-876d-4d88-a39f-487cd1d1ebd4/output/model.tar.gz"
#             }
#         ],
#       "SupportedContentTypes": [ "text/csv" ],
#       "SupportedResponseMIMETypes": [ "text/csv" ],
#     }
# }

In [ ]:
# response = sagemaker_client.create_model_package(
#     **create_model_package_input_dict
# )
# print(json.dumps(response, indent=4))

## Describe Model Package 

In [113]:
sagemaker_client.list_model_packages(
    ModelPackageGroupName="ecg-model-group"
)

{'ModelPackageSummaryList': [{'ModelPackageGroupName': 'ecg-model-group',
   'ModelPackageVersion': 2,
   'ModelPackageArn': 'arn:aws:sagemaker:ap-southeast-1:394599967678:model-package/ecg-model-group/2',
   'CreationTime': datetime.datetime(2023, 6, 1, 7, 53, 38, 912000, tzinfo=tzlocal()),
   'ModelPackageStatus': 'Completed',
   'ModelApprovalStatus': 'PendingManualApproval'},
  {'ModelPackageGroupName': 'ecg-model-group',
   'ModelPackageVersion': 1,
   'ModelPackageArn': 'arn:aws:sagemaker:ap-southeast-1:394599967678:model-package/ecg-model-group/1',
   'CreationTime': datetime.datetime(2023, 6, 1, 7, 38, 14, 418000, tzinfo=tzlocal()),
   'ModelPackageStatus': 'Completed',
   'ModelApprovalStatus': 'PendingManualApproval'}],
 'ResponseMetadata': {'RequestId': '830a2484-3228-49f4-8507-5fd181af7dec',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '830a2484-3228-49f4-8507-5fd181af7dec',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '583',
   'd

In [114]:
sagemaker_client.describe_model_package(
    ModelPackageName="arn:aws:sagemaker:ap-southeast-1:394599967678:model-package/ecg-model-group/1"
)

{'ModelPackageGroupName': 'ecg-model-group',
 'ModelPackageVersion': 1,
 'ModelPackageArn': 'arn:aws:sagemaker:ap-southeast-1:394599967678:model-package/ecg-model-group/1',
 'CreationTime': datetime.datetime(2023, 6, 1, 7, 38, 14, 418000, tzinfo=tzlocal()),
 'InferenceSpecification': {'Containers': [{'Image': '475088953585.dkr.ecr.ap-southeast-1.amazonaws.com/pca:1',
    'ImageDigest': 'sha256:e7f2ee425a9860522b3ffac9992a7292a688ee9fd10c0c645d5b71a13913e789',
    'ModelDataUrl': 's3://sagemaker-ap-southeast-1-394599967678/TrainingJobName27a61f71-876d-4d88-a39f-487cd1d1ebd4/output/model.tar.gz'}],
  'SupportedContentTypes': ['text/csv'],
  'SupportedResponseMIMETypes': ['text/csv']},
 'ModelPackageStatus': 'Completed',
 'ModelPackageStatusDetails': {'ValidationStatuses': [],
  'ImageScanStatuses': []},
 'CertifyForMarketplace': False,
 'ModelApprovalStatus': 'PendingManualApproval',
 'CreatedBy': {'IamIdentity': {'Arn': 'arn:aws:sts::394599967678:assumed-role/TeamRole/MasterKey',
   'Pr

## Deploy a Model from Model Registry 

option 1. after model.create() run model.deploy()

option 2. using boto3 sagemaker client 

In [122]:
endpoint_config_name = "PCAEndpointConfigurationBoto3"
endpoint_name = "PCAEndpointFromModelRegistryBoto3"
model_name = "PCAModelFromModelRegistryBoto3"
model_package_arn = "arn:aws:sagemaker:ap-southeast-1:394599967678:model-package/ecg-model-group/1"

can only create a model for an approved state 

In [117]:
sagemaker_client.update_model_package(
    ModelPackageArn=model_package_arn,
    ModelApprovalStatus="Approved"
)

{'ModelPackageArn': 'arn:aws:sagemaker:ap-southeast-1:394599967678:model-package/ecg-model-group/1',
 'ResponseMetadata': {'RequestId': 'b943cea7-55a7-4d21-a05f-255bd0f5ec04',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'b943cea7-55a7-4d21-a05f-255bd0f5ec04',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '99',
   'date': 'Thu, 01 Jun 2023 08:29:37 GMT'},
  'RetryAttempts': 0}}

In [118]:
response = sagemaker_client.create_model(
    ModelName=model_name,
    ExecutionRoleArn=role,
    Containers=[
        {
            "ModelPackageName": model_package_arn
        }
    ]
)

In [123]:
endpoint_config = sagemaker_client.create_endpoint_config(
    EndpointConfigName=endpoint_config_name,
    ProductionVariants=[
        {
            "InitialInstanceCount": 1,
            "InstanceType": "ml.m4.xlarge",
            "ModelName": model_name,
            "VariantName": "AllTraffic",
        }
    ]
)

In [124]:
response = sagemaker_client.create_endpoint(
    EndpointName=endpoint_name, 
    EndpointConfigName=endpoint_config_name
)

## Test the Endpoint 

In [125]:
response = sagemaker_client.describe_endpoint(
    EndpointName=endpoint_name
)
print(json.dumps(response, indent=4, default=str))

{
    "EndpointName": "PCAEndpointFromModelRegistryBoto3",
    "EndpointArn": "arn:aws:sagemaker:ap-southeast-1:394599967678:endpoint/pcaendpointfrommodelregistryboto3",
    "EndpointConfigName": "PCAEndpointConfigurationBoto3",
    "ProductionVariants": [
        {
            "VariantName": "AllTraffic",
            "DeployedImages": [
                {
                    "SpecifiedImage": "475088953585.dkr.ecr.ap-southeast-1.amazonaws.com/pca@sha256:e7f2ee425a9860522b3ffac9992a7292a688ee9fd10c0c645d5b71a13913e789",
                    "ResolvedImage": "475088953585.dkr.ecr.ap-southeast-1.amazonaws.com/pca@sha256:e7f2ee425a9860522b3ffac9992a7292a688ee9fd10c0c645d5b71a13913e789",
                    "ResolutionTime": "2023-06-01 08:31:18.110000+00:00"
                }
            ],
            "CurrentWeight": 1.0,
            "DesiredWeight": 1.0,
            "CurrentInstanceCount": 1,
            "DesiredInstanceCount": 1
        }
    ],
    "EndpointStatus": "InService",
    "C

In [126]:
runtime = boto3.client("sagemaker-runtime")

In [127]:
response = runtime.invoke_endpoint(
    EndpointName=endpoint_name,
     Body="1,2,3,4\n1,2,3,4\n2,3,4,5",
    ContentType="text/csv"
)
result = json.loads(response['Body'].read())
print(json.dumps(result, indent=4, default=str))

{
    "projections": [
        {
            "projection": [
                1.1226699352264404,
                1.392833948135376,
                -4.927513122558594
            ]
        },
        {
            "projection": [
                1.1226699352264404,
                1.392833948135376,
                -4.927513122558594
            ]
        },
        {
            "projection": [
                1.1157770156860352,
                1.390754222869873,
                -6.926059246063232
            ]
        }
    ]
}
